## Import packages

In [1]:
import pyomo
import pyomo.opt
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
n_time = 24

In [3]:
def _auxDictionary(a):
    temp_dictionary = {}
    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1)] = a[dim0, dim1]
    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1)] = a[dim0]
    return temp_dictionary
#temp_dict1 = _auxDictionary(loadLimit)

## Data

In [4]:
data = {}
data['Inputs'] = pd.read_csv('Inputs.csv')
data['EVs_Inputs'] = pd.read_csv('EVs_Inputs.csv')
data['alpha'] = pd.read_csv('alpha.csv')
data['PchmaxEV'] = pd.read_csv('PchmaxEV.csv')
data['S'] = pd.read_csv('S.csv')

In [5]:
n_evs = data['EVs_Inputs']['Esoc'].size

## Start Time

In [6]:
from datetime import datetime

now = datetime.now()

start_time = now.strftime("%H:%M:%S")
print("Start Time =", start_time)

Start Time = 00:46:20


## Sets

In [7]:
model = pyo.ConcreteModel()

model.ev = pyo.Set(initialize = np.arange(1, n_evs + 1))
model.t = pyo.Set(initialize = np.arange(1, n_time + 1))


## Parameters

In [8]:
model.EEVmax = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,2]))
model.EEVmin = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,1]))
model.ESoc = pyo.Param(model.ev, initialize =_auxDictionary(data['EVs_Inputs'].to_numpy()[:,0]))
model.dT = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,0]))
model.cDA = pyo.Param(model.t, initialize =_auxDictionary(data['Inputs'].to_numpy()[:,1]))
model.PchmaxEV = pyo.Param(model.ev, model.t, initialize =_auxDictionary(data['PchmaxEV'].to_numpy()))
model.S = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['S'].to_numpy()))
model.alpha = pyo.Param(model.ev, model.t, initialize = _auxDictionary(data['alpha'].to_numpy()))
model.RealHour = pyo.Param(model.t, initialize=_auxDictionary(data['Inputs'].to_numpy()[:,2]))
model.penalty1 = 1000
model.penalty2 = 1000
model.penalty3 = 0.6
model.Etrip = pyo.Param(model.ev, initialize=_auxDictionary(data['EVs_Inputs'].to_numpy()[:,3]))
model.n = 0.95
model.m = 0.6
model.DegCost = 0.10
model.factor = 1

## Variables

In [9]:
model.PEV = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.PEVdc = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.EEV = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)
model.Etriprelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Eminsocrelax = pyo.Var(model.ev, model.t, domain = pyo.NonNegativeReals, initialize = 0)
model.Etripn = pyo.Var(model.ev, model.t, domain = pyo.Reals, initialize = 0)

## Constraints

In [10]:
def _balance_etripn(m,ev,t): 
    return m.Etripn[ev,t] == m.Etrip[ev]*m.S[ev,t]/(sum([m.S[ev,k] for k in np.arange(1, n_time + 1)]))
model.balance_etripn = pyo.Constraint(model.ev, model.t, rule = _balance_etripn)

def _balance_energy_EVS3(m,ev,t): 
    if t == 24:
        return m.EEV[ev,t] + m.Etriprelax[ev,t] >= m.EEVmax[ev]*m.m
    return pyo.Constraint.Skip
model.balance_energy_EVS3 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS3)

def _balance_energy_EVS(m,ev,t): 
    if t == 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.ESoc[ev] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS)

def _balance_energy_EVS2(m,ev,t): 
    if t > 1:
        return m.EEV[ev,t] - m.Etriprelax[ev,t] == m.EEV[ev,t-1] + m.PEV[ev,t]*m.dT[t]*m.n - m.PEVdc[ev,t]*m.dT[t]/m.n - m.Etripn[ev,t]
    return pyo.Constraint.Skip
model.balance_energy_EVS2 = pyo.Constraint(model.ev, model.t, rule = _balance_energy_EVS2)

def _power_charging_limit1(m,ev,t): 
    return m.PEV[ev,t] >= 0
model.power_charging_limit1 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit1)

def _power_charging_limit2(m,ev,t): 
    return m.PEV[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_charging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_charging_limit2)

def _power_discharging_limit2(m,ev,t): 
    return m.PEVdc[ev,t] <= m.alpha[ev,t]*m.PchmaxEV[ev,t]*m.factor
model.power_discharging_limit2 = pyo.Constraint(model.ev, model.t, rule = _power_discharging_limit2)

def _energy_limits_EVS_1(m,ev,t): 
    return m.EEVmin[ev] <= m.EEV[ev,t] + m.Eminsocrelax[ev,t]
model.energy_limits_EVS_1 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_1)

def _energy_limits_EVS_2(m,ev,t): 
    return m.EEV[ev,t] <= m.EEVmax[ev]
model.energy_limits_EVS_2 = pyo.Constraint(model.ev, model.t, rule = _energy_limits_EVS_2)  

## Objective function

In [11]:
def _FOag(m):
    return sum([m.PEV[ev,t]*m.dT[t]*m.cDA[t] - m.PEVdc[ev, t]*m.dT[t]*(m.cDA[t]-m.DegCost) + (m.EEVmax[ev] - m.EEV[ev,t])*0.0001 + m.Etriprelax[ev,t]*m.penalty1 + m.Eminsocrelax[ev,t]*m.penalty2 for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

model.FOag = pyo.Objective(rule = _FOag, sense = pyo.minimize)

## Solve the model

In [12]:
from pyomo.opt import SolverFactory

model.write('res_V4_EC.lp',  io_options={'symbolic_solver_labels': True})

# Create a solver
#opt = pyo.SolverFactory('cbc', executable='C:/Program Files/Cbc-releases.2.10.8-x86_64-w64-mingw64/bin/cbc.exe')

opt = pyo.SolverFactory('cplex', executable='C:/Program Files/IBM/ILOG/CPLEX_Studio129/cplex/bin/x64_win64/cplex.exe')
opt.options['LogFile'] = 'res_V4_EC.log'

#opt = pyo.SolverFactory('ipopt', executable='C:/Program Files/Ipopt-3.11.1-win64-intel13.1/bin/ipopt.exe')
#opt.options['print_level'] = 12
#opt.options['output_file'] = "res_V5_EC.log"

results = opt.solve(model)#, tee=True)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: tmpfkibp3jk
  Lower bound: 260563200.91659853
  Upper bound: 260563200.91659853
  Number of objectives: 1
  Number of constraints: 396982
  Number of variables: 338257
  Number of nonzeros: 735238
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: 2.17
  Termination condition: optimal
  Termination message: Primal simplex - Optimal\x3a Objective = 2.6056320092e+08
  Error rc: 0
  Time: 4.529893159866333
# ----------------------------------------------------------
#   Solution Information
# -----------

## Objective Function Value

In [13]:
pyo.value(model.FOag)

260563200.91659954

## End Time

In [14]:
now = datetime.now()

end_time = now.strftime("%H:%M:%S")
print("End Time =", end_time)
print("Dif: {}".format(datetime.strptime(end_time, "%H:%M:%S") - datetime.strptime(start_time, "%H:%M:%S")))

End Time = 00:47:49
Dif: 0:01:29


## Cost Value

In [15]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())
    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)
    return s

In [16]:
PEV_df = ext_pyomo_vals(model.PEV)
dT_df = ext_pyomo_vals(model.dT)
cDA_df = ext_pyomo_vals(model.cDA)
PEVdc_df = ext_pyomo_vals(model.PEVdc)
EEV_df = ext_pyomo_vals(model.EEV)

charge_cost = sum([PEV_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                   for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

discharge_cost = sum([PEVdc_df[t][ev]*dT_df[0][t]*cDA_df[0][t]
                      for ev in np.arange(1, n_evs + 1) for t in np.arange(1, n_time + 1)])

print('Charge cost: {}'.format(charge_cost))
print('Discharge cost: {}'.format(discharge_cost))

Charge cost: 5156276.7631342625
Discharge cost: 3758079.722012131


## Results

In [17]:
print("Total Charge: {}".format(np.sum(PEV_df.to_numpy())))

Total Charge: 37367844.26556182


In [18]:
print("Total Discharge: {}".format(np.sum(PEVdc_df.to_numpy())))

Total Discharge: 13631047.232543018


In [19]:
EEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,38720.494033,38720.494033,38720.494033,38720.494033,38720.494033,38720.494033,38720.494033,38720.494033,38720.494033,38720.494033,...,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.0
2,57000.000000,57000.000000,55176.484860,55176.484860,55176.484860,55176.484860,58596.484860,62016.484860,65436.484860,68856.484860,...,76351.924253,73587.363645,70822.803038,68058.242430,68058.242430,65293.681823,62529.121215,59764.560608,57000.000000,57000.0
3,38380.759199,38380.759199,38380.759199,38380.759199,38380.759199,38380.759199,38380.759199,38380.759199,38380.759199,38380.759199,...,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.000000,40800.0
4,57000.000000,50837.846278,43258.898910,43258.898910,43258.898910,43258.898910,50098.898910,56938.898910,63778.898910,66079.265940,...,61539.632970,61539.632970,61539.632970,61539.632970,61539.632970,61539.632970,61539.632970,61539.632970,61539.632970,57000.0
5,8081.032174,5289.473684,1500.000000,1500.000000,1500.000000,1500.000000,4920.000000,8118.967826,8118.967826,8118.967826,...,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,48000.000000,48000.000000,48000.000000,51420.000000,54840.000000,55230.025760,58650.025760,62070.025760,65490.025760,61940.017173,...,54970.008587,54970.008587,54970.008587,54970.008587,54970.008587,54970.008587,54970.008587,54970.008587,54970.008587,48000.0
2346,40594.658232,40594.658232,40594.658232,40594.658232,40594.658232,40594.658232,40594.658232,40594.658232,40594.658232,40594.658232,...,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.000000,42600.0
2347,8088.619305,8088.619305,1500.000000,1500.000000,1500.000000,1500.000000,8111.380695,8111.380695,8111.380695,8111.380695,...,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.000000,8100.0
2348,8100.000000,8100.000000,1500.000000,1500.000000,1500.000000,1500.000000,8340.000000,8399.691647,8399.691647,8399.691647,...,8249.845824,8249.845824,8249.845824,8249.845824,8249.845824,8249.845824,8249.845824,8249.845824,8249.845824,8100.0


In [20]:
PEV_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0,...,4377.907298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,3600.0000,3600.000000,3600.0,3600.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0,...,5093.138529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,7200.0000,7200.000000,7200.0,7200.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.000000,3600.0000,3367.334553,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,0.0,0.0,0.0,3600.0,3600.0,410.553432,3600.0000,3600.000000,3600.0,3600.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2346,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0,...,4221.772142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2347,0.0,0.0,0.0,0.0,0.0,0.000000,6959.3481,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2348,0.0,0.0,0.0,0.0,0.0,0.000000,7200.0000,62.833313,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
PEVdc_df

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
1,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,1732.339383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5854.046036,7200.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,2651.980566,3600.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2346,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2347,0.0,0.000000,6259.188339,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2348,0.0,0.000000,6270.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
PEV_df.sum()

1     1.025082e+04
2     9.672625e+03
3     9.389474e+03
4     3.749835e+05
5     1.655207e+05
6     6.452560e+04
7     7.811251e+06
8     5.498543e+06
9     4.827462e+06
10    3.805512e+06
11    1.986856e+06
12    1.267546e+06
13    9.170936e+05
14    8.557098e+06
15    1.665203e+06
16    1.291361e+04
17    0.000000e+00
18    0.000000e+00
19    0.000000e+00
20    1.383817e+04
21    0.000000e+00
22    0.000000e+00
23    1.786655e+05
24    1.915192e+05
dtype: float64

In [23]:
PEVdc_df.sum()

1     2.671488e+06
2     4.120448e+06
3     6.839111e+06
4     0.000000e+00
5     0.000000e+00
6     0.000000e+00
7     0.000000e+00
8     0.000000e+00
9     0.000000e+00
10    0.000000e+00
11    0.000000e+00
12    0.000000e+00
13    0.000000e+00
14    0.000000e+00
15    0.000000e+00
16    0.000000e+00
17    0.000000e+00
18    0.000000e+00
19    0.000000e+00
20    0.000000e+00
21    0.000000e+00
22    0.000000e+00
23    0.000000e+00
24    0.000000e+00
dtype: float64

In [24]:
Etriprelax_df = ext_pyomo_vals(model.Etriprelax)

In [25]:
Etriprelax_df.sum()

1          0.000000
2          0.000000
3          0.000000
4          0.000000
5          0.000000
6          0.000000
7          0.000000
8          0.000000
9          0.000000
10         0.000000
11         0.000000
12         0.000000
13      4822.572516
14     11820.087758
15      8136.712526
16     12361.970335
17      3113.189245
18      5907.229262
19         0.000000
20      5454.472282
21      4363.416847
22         0.000000
23         0.000000
24    201677.048084
dtype: float64

In [26]:
Eminsocrelax_df = ext_pyomo_vals(model.Eminsocrelax)

In [27]:
Eminsocrelax_df.sum()

1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
dtype: float64

In [28]:
Etripn_df = ext_pyomo_vals(model.Etripn)

In [29]:
Etripn_df.sum()

1     1.100987e+06
2     2.849494e+04
3     9.544837e+03
4     0.000000e+00
5     8.494229e+03
6     0.000000e+00
7     9.425414e+03
8     7.520691e+03
9     0.000000e+00
10    7.243526e+05
11    1.681718e+06
12    0.000000e+00
13    8.133524e+05
14    3.464170e+06
15    1.548599e+06
16    1.564664e+06
17    4.420251e+05
18    9.451253e+05
19    3.061923e+05
20    1.061668e+06
21    1.214125e+06
22    1.112450e+06
23    1.203906e+06
24    4.363501e+06
dtype: float64

## Save results in csv files

In [30]:
import os 
folder = 'results_ToU_V2G_' + str(n_evs)

if not os.path.exists(folder):
    os.makedirs(folder)
    
EEV_df.to_csv(folder + '/EEV.csv')
PEV_df.to_csv(folder + '/PEV.csv')
PEVdc_df.to_csv(folder + '/PEVdc.csv')
PEV_df.sum().to_csv(folder + '/PEV_h.csv')
PEVdc_df.sum().to_csv(folder + '/PEVdc_h.csv')

Etriprelax_df.to_csv(folder + '/Etriprelax.csv')
Etriprelax_df.sum().to_csv(folder + '/Etriprelax_h.csv')

Eminsocrelax_df.to_csv(folder + '/Eminsocrelax.csv')
Eminsocrelax_df.sum().to_csv(folder + '/Eminsocrelax_h.csv')

Etripn_df.to_csv(folder + '/Etripn.csv')
Etripn_df.sum().to_csv(folder + '/Etripn_h.csv')